In [ ]:
# Bibliotecas Necessárias
!sudo apt update -y
!sudo apt install -y tesseract-ocr poppler-utils tesseract-ocr-por
!pip install -q pytesseract pdfplumber pdf2image Pillow

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,572 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5,155 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd

In [ ]:
# Imports Necessários
import os
from PIL import Image
import pytesseract
import pdfplumber
from pdf2image import convert_from_path
import re

In [ ]:
# Configuração Inicial
diretorio_projeto = "Nuvem-Desafio-Tecnico-IA"

diretorio_documentos = os.path.join(diretorio_projeto, "data", "documentos_originais")

os.makedirs(diretorio_documentos, exist_ok = True)

diretorio_textos = os.path.join(diretorio_projeto, "data", "extracted_texts")

os.makedirs(diretorio_textos, exist_ok = True)


In [ ]:
# Funções de Extração

def extrair_texto_pdf(arquivo_pdf):
  """
  Extrai o texto de um arquivo PDF
  """
  texto_extraido = ""
  try:
    with pdfplumber.open(arquivo_pdf) as pdf:
      print(f"Extraindo texto do arquivo: {arquivo_pdf}")
      for pagina in pdf.pages:
        texto_pagina = pagina.extract_text()
        if texto_pagina:
          texto_extraido += texto_pagina + "\n"

    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo do PDF '{arquivo_pdf}")

  except Exception as e:
    print(f"Erro ao extrair texto do PDF: {e}")
    texto_extraido = ""

  return texto_extraido


def aplicar_OCR_imagem(arquivo_imagem):
  """
  Aplica OCR em uma imagem e retorna o texto extraído
  """
  texto_extraido = ""
  try:
    print(f"Aplicando OCR na imagem: {arquivo_imagem}")
    imagem = Image.open(arquivo_imagem)
    texto_extraido = pytesseract.image_to_string(imagem, lang = 'por')

    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo da imagem '{arquivo_imagem}'")

  except Exception as e:
    print(f"Erro ao aplicar OCR na imagem: {arquivo_imagem}: {e}")
    texto_extraido = ""

  return texto_extraido

def aplicar_OCR_pdf(arquivo_pdf):
  """
  Aplica OCR em um PDF só de imagens e retorna o texto extraído
  """
  texto_extraido = ""
  try:
    print(f"Aplicando OCR no PDF: {arquivo_pdf}")
    imagens = convert_from_path(arquivo_pdf, dpi = 300)
    for i, imagem in enumerate(imagens):
      texto_pagina = pytesseract.image_to_string(imagem, lang = 'por')
      if texto_pagina:
        texto_extraido += texto_pagina + "\n"
    if not texto_extraido.strip():
      print(f"Aviso: Pouco ou nenhum texto nativo do PDF '{arquivo_pdf}'")

  except Exception as e:
    print(f"Erro ao aplicar OCR no PDF: {e}")
    texto_extraido = ""
  return texto_extraido


In [ ]:
# Função Princuipal

def extrair_texto_de_arquivos(arquivo):
  """
  Extrai o texto de arquivos PDF ou Imagens. Usando OCR se necessário
  """
  texto_extraido = ""
  extencao = os.path.splitext(arquivo)[1].lower()
  print(f"\n--- Processando documento: {arquivo} ---")

  if extencao == ".pdf":
    print("Extraindo texto de arquivo PDF...")
    texto_extraido = extrair_texto_pdf(arquivo)

    if not texto_extraido.strip or len(texto_extraido.strip()) < 20:
      print("Texto extraido do arquivo pequeno ou vazio, tentando aplicar OCR...")
      texto_extraido_ocr = aplicar_OCR_pdf(arquivo)
      if texto_extraido_ocr.strip():
        texto_extraido = texto_extraido_ocr
      else:
        print("Nenhum texto extraido com OCR.")

  elif extencao in [".jpg", ".jpeg", ".png", ".webp"]:
    texto_extraido = aplicar_OCR_imagem(arquivo)

  else:
    print(f"Extensão de arquivo não suportada: {extencao}")

  if not texto_extraido:
    print(f"AVISO FINAL: Nenhum texto significativo extraído de {arquivo}.")

  else:
    print(f"Texto extraido de {arquivo}:")
    print(f"Número de caracteres extraidos: {len(texto_extraido)}")

  return texto_extraido


def salvar_textos(textos_extraidos, saida):
  os.makedirs(saida, exist_ok = True)
  for arq , texto in textos_extraidos.items():
    nome_original = os.path.basename(arq)
    nome_saida = os.path.join(saida, f"{os.path.splitext(nome_original)[0]}.txt")
    with open(nome_saida, 'w', encoding = 'utf-8') as arquivo:
      arquivo.write(texto)
    print(f"Texto salvo de {nome_original} em: {nome_saida}")

In [ ]:
# Prepara os .txts

documentos_para_processar = []

for arquivos in os.listdir(diretorio_documentos):
  if arquivos.lower().endswith(('.pdf', '.jpg', '.jpeg', '.png', '.webp')):
    caminho_arquivo = os.path.join(diretorio_documentos, arquivos)
    documentos_para_processar.append(caminho_arquivo)

if not documentos_para_processar:
  print("Nenhum arquivo PDF, JPG, JPEG, PNG ou WEBP encontrado na pasta.")
else:
  print("Arquivos PDF, JPG, JPEG, PNG e WEBP encontrados:")
  for arquivo in documentos_para_processar:
    print(f"- {os.path.basename(arquivo)}")

textos_extraidos = {}

for docs in documentos_para_processar:
  if os.path.exists(docs):
    print(f"Processando documento: {docs}")
    texto_extraido = extrair_texto_de_arquivos(docs)
    if texto_extraido:
      textos_extraidos[docs] = texto_extraido
  else:
    print(f"Documento não encontrado em {docs}. Faça o upload ou confirme o path")


salvar_textos(textos_extraidos, diretorio_textos)


Arquivos PDF, JPG, JPEG, PNG e WEBP encontrados:
- CÓDIGO DE OBRAS.pdf
- tabela.jpeg
Processando documento: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/CÓDIGO DE OBRAS.pdf

--- Processando documento: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/CÓDIGO DE OBRAS.pdf ---
Extraindo texto de arquivo PDF...
Extraindo texto do arquivo: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/CÓDIGO DE OBRAS.pdf
Texto extraido de Nuvem-Desafio-Tecnico-IA/data/documentos_originais/CÓDIGO DE OBRAS.pdf:
Número de caracteres extraidos: 204239
Processando documento: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/tabela.jpeg

--- Processando documento: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/tabela.jpeg ---
Aplicando OCR na imagem: Nuvem-Desafio-Tecnico-IA/data/documentos_originais/tabela.jpeg
Texto extraido de Nuvem-Desafio-Tecnico-IA/data/documentos_originais/tabela.jpeg:
Número de caracteres extraidos: 2446
Texto salvo de CÓDIGO DE OBRAS.pdf em: Nuvem-Desafio-Tecnico-IA/data